# Explore Methods

In this notebook, we will explore different methods for analyzing paths between airports. Mainly, we will find out in which ways graph-based approaches are preferable to other commonly used methods. <br>
Note that this notebook is not meant to be a standalone tutorial. Please read the corresponding medium article linked in the repository.

In [7]:
import pandas as pd
import numpy as np

import timeit

## Load Data

The dataset we load here is already cleaned. See `clean_dataset.ipynb` for more.

In [16]:
df = pd.read_csv("flights_sample.csv", index_col = 0)
print(df.shape)
df.head(10)

(1000, 2)


,origin,destination
day,,
2019-01-16 00:00:00+00:00,KEWR,KDFW
2019-01-01 00:00:00+00:00,KONT,KPHX
2019-01-09 00:00:00+00:00,LSZH,LTBA
2019-01-15 00:00:00+00:00,KSFO,KSAN
2019-01-01 00:00:00+00:00,PANC,MS65
2019-01-01 00:00:00+00:00,EBAW,EBMB
2019-01-09 00:00:00+00:00,KSLC,KOAK
2019-01-21 00:00:00+00:00,KDEN,KAUS
2019-01-22 00:00:00+00:00,KLAX,KRNO


## Create Adjacency Matrix

In [19]:
# Get unique airports
airports = np.unique(np.append(df["origin"], df["destination"]))
print(airports.shape)
airports[:20]

(664,)


array(['01FA', '02XS', '06FA', '06TE', '0GA1', '0GA2', '0NY0', '0WI5',
       '13AZ', '14FA', '16FA', '16TX', '1AZ2', '1CO4', '1LA1', '1OR3',
       '1WI6', '1XS1', '20GA', '20II'], dtype=object)

In [22]:
# Map airports in dataframe to their graph number
mapping_dict = {k:i for i,k in enumerate(airports)}

df_mapped = df.applymap(lambda x: mapping_dict[x])
print(df_mapped.shape)
df_mapped.head(10)

(1000, 2)


,origin,destination
day,,
2019-01-16 00:00:00+00:00,284,272
2019-01-01 00:00:00+00:00,372,385
2019-01-09 00:00:00+00:00,522,525
2019-01-15 00:00:00+00:00,415,408
2019-01-01 00:00:00+00:00,572,537
2019-01-01 00:00:00+00:00,94,98
2019-01-09 00:00:00+00:00,419,368
2019-01-21 00:00:00+00:00,271,236
2019-01-22 00:00:00+00:00,331,403


In [23]:
# Create nxn-matrix of zeros
A = np.zeros((len(airports), len(airports)), dtype = int)
print("Shape:",A.shape)

# Fill adjacency matrix
for date, flight in df_mapped.iterrows():
    i, j = flight["origin"], flight["destination"]
    if A[i,j] == 0:
        A[i,j], A[j,i] = 1,1

A[:20,:20]

Shape: (664, 664)


array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0,

## Analysis 1: Can you fly from A to B directly?

### Conventional Pandas Method

In [24]:
def exists_direct_path(df, node1, node2):
    
    # Loop through each row
    for i, row in df.iterrows():
        
        # Check whether (node1, node2) or (node2, node1) is in the dataset
        if {row["origin"], row["destination"]} == {node1,node2}:
            return True
        
    return False

In [25]:
node1, node2 = "KEWR", "KDFW"
exists_direct_path(df, node1, node2)

True

### Graph Method

In [17]:
node1, node2 = mapping_dict["KEWR"], mapping_dict["KDFW"]
A[node1, node2] == 1

True

## Analysis 2: How many destinations is A directly linked to?

### Conventional Pandas Method

In [26]:
def degree(df, node):
    
    # Setup empty list
    flights = []
    
    # Loop through every row
    for i, row in df.iterrows():
        
        # If the node is either an origin or a destination, there must be a direct path
        if row["origin"] == node:
            flights.append(row["destination"])
        elif row["destination"] == node:
            flights.append(row["origin"])
    
    # Remove duplicates
    flights = list(set(flights))
    
    return len(flights)

In [27]:
node = "KEWR"
degree(df, node)

13

## Graph Method

In [28]:
node = mapping_dict["KEWR"]
A[node].sum()

13

## Analysis 3 : Shortest Path from A to B

### Graph Method

In [30]:
def shortest_path(A, x, y, iterations = 10, mapping_dict = mapping_dict):
    
    # Create copy of Matrix to work on
    M = A.copy()
    
    # Get numerical representations of airports
    i, j = mapping_dict[x], mapping_dict[y]
    
    # Define the maximum power to which A is to be raised
    iterations = 10

    # A = A*A until a path is found or the max iterations are reached
    for k in range(iterations):
        if M[i,j] == 0:
            M = np.matmul(M,A)
        else:
            return (x,y,k)

In [31]:
home_airport = "LFPO"
vacation_destinations = ["YWHA", "LTAC", "LIRF", "EVRA", "KRFI"]
max_changes = 2

n_changes = [shortest_path(A, home_airport, destination) for destination in vacation_destinations]
acceptable = [destination for location, destination, changes in n_changes if changes <= 2]
acceptable

['LIRF', 'EVRA']